In [90]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
from modules import *

In [91]:
data = pd.read_csv("술 리뷰(전체).csv", encoding='utf-8-sig')
data.drop_duplicates(['text'],ignore_index=True, inplace=True) # 중복 제거
# data.columns = ['date', 'star','doc']
# del data['Unnamed: 0']
data

,date,star,text,name
0,2022-10-31,5,지인으로부터 알게된 복순도가 막걸리처음 먹는순간 너무 맛있어서 깜짝놀랐어요원래 술을...,복순도가
1,2022-11-17,5,그냥 차게 마셔도 군내나 잔여물 찌꺼기 느낌이 거의 없이 깔끔담백하고 샴페인처럼 톡...,복순도가
2,2022-10-19,5,지인이 비싼데 고급지고 맛있는 막걸리가있다고 해서하도 궁금하여 한번 시켜봤어요배송도...,복순도가
3,2022-10-28,3,본인은 맛있었는데 아버지는 너무 삭힌거같다는 평이네요 새콤함이 좀 있긴함근데 막걸리...,복순도가
4,2022-10-13,5,복순도가 맛있다고 한참전에 소문만 들었지 먹어본것은 올해가 처음이네요 평소에도 달달...,복순도가
...,...,...,...,...
32093,2022-11-28,5,벌써 두번째 주문입니다 김장때 수육이랑 같이 먹었는데 너무 맛있었고 어른들이 특히 ...,복순도가
32094,2022-11-28,5,술 한잔도 못드시는 어머니도 너무 맛있게 잘 드시네요,복순도가
32095,2022-11-28,5,역시 최고에요최고 최고,복순도가
32096,2022-11-28,5,맛ㅇㅅ어요 집에서 싸게 주문해서 먹어서 맛있어요ㅔ,복순도가


In [92]:
data.text = data.text.str.replace("[^가-힣 ]"," ") # 한글만 남기고 특수문자, 숫자, 영어 삭제
data.text = data.text.str.replace("\s+", " ") # white space 삭제
data.text

0        지인으로부터 알게된 복순도가 막걸리처음 먹는순간 너무 맛있어서 깜짝놀랐어요원래 술을...
1        그냥 차게 마셔도 군내나 잔여물 찌꺼기 느낌이 거의 없이 깔끔담백하고 샴페인처럼 톡...
2        지인이 비싼데 고급지고 맛있는 막걸리가있다고 해서하도 궁금하여 한번 시켜봤어요배송도...
3        본인은 맛있었는데 아버지는 너무 삭힌거같다는 평이네요 새콤함이 좀 있긴함근데 막걸리...
4        복순도가 맛있다고 한참전에 소문만 들었지 먹어본것은 올해가 처음이네요 평소에도 달달...
                               ...                        
32093    벌써 두번째 주문입니다 김장때 수육이랑 같이 먹었는데 너무 맛있었고 어른들이 특히 ...
32094                        술 한잔도 못드시는 어머니도 너무 맛있게 잘 드시네요
32095                                         역시 최고에요최고 최고
32096                           맛 어요 집에서 싸게 주문해서 먹어서 맛있어요 
32097                        와 인생막걸리입니다 이제 다른거 못먹어요복순도가 최고
Name: text, Length: 32098, dtype: object

In [93]:
# 검토 후 추가 삭제 문구 적용
data.dropna(inplace=True)

In [94]:
data[data.text.isnull()]

,date,star,text,name


In [95]:
# data = data[data.doc.str.contains('맛|향|달|넘김|부드|단|도수|냄세')]
# data.reset_index(drop=True, inplace=True)

In [96]:
len(data)

32097

In [97]:
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk import word_tokenize
from konlpy.tag import Okt

In [98]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
from konlpy.tag import Okt #형태소 분석
from nltk import bigrams
from nltk.util import ngrams
import pandas as pd

from gensim.models.ldamodel import LdaModel
import matplotlib.pyplot as plt
from gensim.models.coherencemodel import CoherenceModel # 토픽 개수 검증
from tqdm import tqdm
# from tqdm.notebook import tqdm
from gensim import corpora
from gensim.models import TfidfModel

import warnings
warnings.filterwarnings(action='ignore')

from modules import *

In [99]:
#원하는 태그 추출 & 불용어 제거
okt = Okt()

#불용어
stopwords = pd.read_csv('ko-stopwords.csv') #한국어불용어사전 불러오기
stopwords=list(stopwords['stopwords']) 
stopwords.extend(['에서','고','이다','는','한','씨', "것","거","게","데","이다","건","고","되다","되어다","걸","기",
                  "시","네","듯","랍니","중이","얘","스","도도", "나","수","개","내","기","제","저","인","있다","이렇다",
                  "그렇다","번","위","팅","분","인","링","란","포","두", "진짜", "하다" ,"이다" ,"가다", "이제" ,"들다",
                 '에서','고','이다','쓰리다 도어즈','일요일','휴무','수표','층','틀다','차차','드므','맛집','술집','길','가가'
                 '층','개다 성','사진','파다','동영상','아스 론','자다','는','희다 스토리','한','씨', "것","거","게","데",
                 "이다","건","고","되다","되어다","걸","기", "시","네","듯","랍니","중이","얘","스","도도", "나","수","개",
                 "내","기","제","저","인","있다","이렇다", "그렇다","번","위","팅","분","인","링","란","포","두", "진짜",
                 "하다" ,"이다" ,"가다", "이제" ,"들다", "먹다", "보다"]) #불용어 추가

stopwords=set(stopwords) # 중복제거    

# 형태소 분석 함수 만들기
def okt_pos_tagging2(string):
    pos_words = okt.pos(string, stem=True, norm=True) # 형태소 분석. 단어는 사전형으로 바꿔주기
    words = [word for word, tag in pos_words if tag 
             in ['Noun', 'Adjective', 'Verb','Adverb'] if word not in stopwords ]
    # word2=[]
    # word2.append(words)
    # bigr = bigram_(word2)
    # asd = []
    
    # for i in range(len(bigr[2])):
    #     asd.append(bigr[2][i])
    return words

# def bigram_(tokens):
    
#     #document 별 형태소분석기로 분리된 토큰들을 bigram으로 변환(토큰은 string 상태)
#     bigram_stack = [] #변환된 bigram
#     for token in tokens:
#         bigram = bigrams(token)
#         bigram_token = [' '.join(grams) for grams in bigram]
#         bigram_stack.append(bigram_token)
        
#     #bigram  딕셔너리 생성(아이디 매칭)
#     id2word = corpora.Dictionary(bigram_stack) #unigram
#     #id2word.token2id  #아이디가 매칭된 딕셔너리 확인용

#     #딕셔너리에 매칭한 corpus 생성
#     #생성한 bigram 딕셔너리에서 bigram_stack의 토큰을 아이디로 매칭변환
#     texts = bigram_stack 
#     corpus = [id2word.doc2bow(text) for text in texts]
    
#     #gensim으로 tf-idf 처리 #바이그램 요소 하나를 하나의 워드로 처리해서 tf_idf 점수
#     tfidf = TfidfModel(corpus) 
#     corpus_tfidf = tfidf[corpus]
#     tfidf_corpus = [x for x in corpus_tfidf]
#     return tfidf_corpus, corpus ,id2word

In [100]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(tokenizer = lambda x: okt_pos_tagging2(x))
bow_vect = vect.fit_transform(data['text'].tolist())
word_list = vect.get_feature_names()
count_list = bow_vect.toarray().sum(axis=0)

In [131]:
bow_vect.shape

(32097, 9038)

In [132]:
word_count_dict = dict(zip(word_list, count_list))
word_count_dict


{'가가': 61,
 '가감': 1,
 '가걱': 1,
 '가게': 26,
 '가격': 1336,
 '가격떄문': 1,
 '가계': 1,
 '가공': 1,
 '가구': 2,
 '가글': 1,
 '가급적': 1,
 '가기': 8,
 '가까워지다': 2,
 '가까이': 9,
 '가깝다': 31,
 '가끔': 154,
 '가나': 2,
 '가난': 1,
 '가난하다': 1,
 '가내': 1,
 '가늘다': 1,
 '가능': 7,
 '가능성': 2,
 '가능하다': 55,
 '가대': 1,
 '가도': 11,
 '가동': 1,
 '가두': 1,
 '가두다': 1,
 '가득': 48,
 '가득가득': 1,
 '가득하다': 58,
 '가라': 12,
 '가라앉다': 10,
 '가라앉히다': 3,
 '가로': 36,
 '가루': 4,
 '가르다': 94,
 '가리다': 6,
 '가리비': 1,
 '가마': 1,
 '가막': 1,
 '가만': 39,
 '가만히': 2,
 '가면': 11,
 '가물': 12,
 '가미': 7,
 '가바': 1,
 '가방': 9,
 '가버리다': 4,
 '가벼워지다': 1,
 '가볍다': 327,
 '가보': 1,
 '가보다': 32,
 '가복': 6,
 '가봉': 1,
 '가분': 1,
 '가뿐하다': 1,
 '가사': 4,
 '가성': 49,
 '가세': 1,
 '가쇼': 1,
 '가수': 1,
 '가스': 17,
 '가시': 5,
 '가시다': 5,
 '가신': 2,
 '가야': 3,
 '가오니': 1,
 '가온': 1,
 '가요': 37,
 '가요제': 1,
 '가운': 1,
 '가운데': 2,
 '가위': 1,
 '가은': 1,
 '가을': 3,
 '가의': 27,
 '가이거': 2,
 '가이드': 1,
 '가이드북': 1,
 '가인': 1,
 '가작': 1,
 '가장': 83,
 '가절': 1,
 '가정': 3,
 '가제': 4,
 '가져가다': 72,
 '가져다주다': 1,
 '가져오다': 5,
 '가조': 1,
 '가족': 611

In [133]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_vectorizer = TfidfTransformer()
tf_idf_vect = tfidf_vectorizer.fit_transform(bow_vect)

In [134]:
print(tf_idf_vect.shape)


(32097, 9038)


In [135]:
print(tf_idf_vect[0])

  (0, 9015)	0.13717171389621002
  (0, 8842)	0.15662121550967673
  (0, 8045)	0.11580400421880917
  (0, 8014)	0.172305571400173
  (0, 7949)	0.10694407912280997
  (0, 7543)	0.17572441783408224
  (0, 7498)	0.1481065073269119
  (0, 7280)	0.12603385397890784
  (0, 7252)	0.18245098348207195
  (0, 7084)	0.11542198138068736
  (0, 7026)	0.06358718408533923
  (0, 7016)	0.04558144862234138
  (0, 6839)	0.08082589665627679
  (0, 6540)	0.19533214895003315
  (0, 6134)	0.1447296772153638
  (0, 6013)	0.11659188890784285
  (0, 5870)	0.1317787008228783
  (0, 5743)	0.1232702044382649
  (0, 5730)	0.11963920037525672
  (0, 5533)	0.14303923057494333
  (0, 5232)	0.1465852959657297
  (0, 5224)	0.15662121550967673
  (0, 5203)	0.11038490485162401
  (0, 5202)	0.15168639835183234
  (0, 5083)	0.1647447874192643
  :	:
  (0, 4584)	0.1724506502885314
  (0, 4274)	0.1874811617485097
  (0, 4011)	0.10002076434923599
  (0, 3948)	0.11031250477825545
  (0, 3636)	0.07303345051071573
  (0, 2979)	0.07790720546651468
  (0, 2733)	

In [136]:
invert_index_vectorizer = {v: k for k, v in vect.vocabulary_.items()}
print(str(invert_index_vectorizer)[:100]+'...')

{7252: '지인', 5203: '알', 3636: '복', 4622: '순도', 2573: '막걸리', 7543: '처음', 4620: '순간', 1366: '너무', 2733...


In [138]:
def rating_to_label(rating):
    if rating > 3:
        return 1
    else:
        return 0

data['y'] = data['star'].apply(lambda x : rating_to_label(x))
len(data[data['y']==0])

1756

In [139]:
from sklearn.model_selection import train_test_split

x = tf_idf_vect
y = data['y']

# positive_random_idx = data[data['y']==1].sample(48, random_state=12).index.tolist()
# negative_random_idx = data[data['y']==0].sample(48, random_state=12).index.tolist()

# random_idx = positive_random_idx + negative_random_idx
# x = tf_idf_vect[random_idx]
# y = data['y'][random_idx]


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state=1)


In [140]:
x.shape,y.shape

((32097, 9038), (32097,))

In [141]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# fit in training set
lr = LogisticRegression()
lr.fit(x_train, y_train)

# predict in test set
y_pred = lr.predict(x_test)

In [142]:
print('accuracy: %.4f' % accuracy_score(y_test, y_pred))
print('precision: %.4f' % precision_score(y_test, y_pred))
print('recall: %.4f' % recall_score(y_test, y_pred))
print('F1: %.4f' % f1_score(y_test, y_pred))


accuracy: 0.9474
precision: 0.9503
recall: 0.9965
F1: 0.9729


In [143]:
coef_pos_index = sorted(((value, index) for index, value in enumerate(lr.coef_[0])), reverse = True)
coef_neg_index = sorted(((value, index) for index, value in enumerate(lr.coef_[0])), reverse = False)
coef_pos_index



[(5.3667397373501595, 2733),
 (3.1367610783615607, 7016),
 (2.6540613082481213, 7662),
 (2.3445223183410286, 6483),
 (2.3165211334426488, 3711),
 (2.068515322758357, 1092),
 (2.0301695942407236, 5583),
 (1.988571775713822, 2629),
 (1.8692740429287265, 6718),
 (1.517709416581199, 4333),
 (1.467167430487545, 6542),
 (1.4341408326601826, 3636),
 (1.334242308401297, 4622),
 (1.3329448965746715, 8629),
 (1.3319689696501427, 5769),
 (1.2969687225056332, 3701),
 (1.2026690844360965, 1080),
 (1.1639395922572309, 781),
 (1.1589842567427813, 4384),
 (1.1384165722174036, 2099),
 (1.1298491026017128, 6901),
 (1.0943694023388828, 435),
 (1.0942978826991965, 7677),
 (1.0822627941027976, 5125),
 (1.0725622270268933, 1024),
 (1.04875041792926, 1180),
 (1.042007278382733, 5183),
 (1.0407294915920675, 4815),
 (1.0356842400175708, 1388),
 (1.0343665667059099, 1736),
 (1.0296754855257197, 7850),
 (0.9955554352501623, 8094),
 (0.9918249571738277, 2532),
 (0.9898510313181532, 7140),
 (0.989080888700692, 461

In [156]:
coef_list = []
coef_list2 = []
for coef in coef_pos_index[:20]:
    print(invert_index_vectorizer[coef[1]], coef[0])
    coef_list.append(invert_index_vectorizer[coef[1]])
    coef_list2.append(coef[0])
coef_list

맛있다 5.3667397373501595
좋다 3.1367610783615607
최고 2.6540613082481213
잇다 2.3445223183410286
부드럽다 2.3165211334426488
꼼꼼하다 2.068515322758357
역시 2.0301695942407236
만족하다 1.988571775713822
적당하다 1.8692740429287265
선물 1.517709416581199
자주 1.467167430487545
복 1.4341408326601826
순도 1.334242308401297
항상 1.3329448965746715
와인 1.3319689696501427
부담 1.2969687225056332
꼭 1.2026690844360965
귀엽다 1.1639395922572309
성공하다 1.1589842567427813
드디어 1.1384165722174036


['맛있다',
 '좋다',
 '최고',
 '잇다',
 '부드럽다',
 '꼼꼼하다',
 '역시',
 '만족하다',
 '적당하다',
 '선물',
 '자주',
 '복',
 '순도',
 '항상',
 '와인',
 '부담',
 '꼭',
 '귀엽다',
 '성공하다',
 '드디어']

In [157]:
c_non_list = []
c_non_list2 = []
for coef in coef_neg_index[:20]:
    print(invert_index_vectorizer[coef[1]], coef[0])
    c_non_list.append(invert_index_vectorizer[coef[1]])
    c_non_list2.append(coef[0])
c_non_list

아니다 -3.6583655337347367
별로 -3.384682447818589
시큼하다 -3.178910127716295
그냥 -3.107501772458313
셔 -3.0938184757800875
밍밍 -2.958018521772605
실망 -2.768677659215181
식초 -2.6808566070973003
신맛 -2.654456002438325
맛없다 -2.619518718624531
달다 -2.2864575155987485
기대하다 -2.237880283516813
루 -2.181831794021783
상한 -2.0607120690898144
입맛 -2.0020818253609023
대비 -1.9463209534590393
상 -1.9076334656271718
물탄 -1.822952527050014
밋밋하다 -1.7798379800097348
심하다 -1.765574944919876


['아니다',
 '별로',
 '시큼하다',
 '그냥',
 '셔',
 '밍밍',
 '실망',
 '식초',
 '신맛',
 '맛없다',
 '달다',
 '기대하다',
 '루',
 '상한',
 '입맛',
 '대비',
 '상',
 '물탄',
 '밋밋하다',
 '심하다']

In [161]:
df = pd.Series(coef_list)
df1 = pd.Series(coef_list2)
df2 = pd.Series(c_non_list)
df3 = pd.Series(c_non_list2)
df_new = pd.concat([df,df1,df2,df3], axis=1)
df_new

,0,1,2,3
0,맛있다,5.366740,아니다,-3.658366
1,좋다,3.136761,별로,-3.384682
2,최고,2.654061,시큼하다,-3.178910
3,잇다,2.344522,그냥,-3.107502
4,부드럽다,2.316521,셔,-3.093818
5,꼼꼼하다,2.068515,밍밍,-2.958019
6,역시,2.030170,실망,-2.768678
7,만족하다,1.988572,식초,-2.680857
8,적당하다,1.869274,신맛,-2.654456
9,선물,1.517709,맛없다,-2.619519


In [163]:
df_new.to_csv('포지셔닝.csv')

In [146]:
for coef in coef_neg_index[:]:
    print(invert_index_vectorizer[coef[1]], coef[0])

아니다 -3.6583655337347367
별로 -3.384682447818589
시큼하다 -3.178910127716295
그냥 -3.107501772458313
셔 -3.0938184757800875
밍밍 -2.958018521772605
실망 -2.768677659215181
식초 -2.6808566070973003
신맛 -2.654456002438325
맛없다 -2.619518718624531
달다 -2.2864575155987485
기대하다 -2.237880283516813
루 -2.181831794021783
상한 -2.0607120690898144
입맛 -2.0020818253609023
대비 -1.9463209534590393
상 -1.9076334656271718
물탄 -1.822952527050014
밋밋하다 -1.7798379800097348
심하다 -1.765574944919876
이하 -1.747625045275176
개취 -1.7426300550732226
강하다 -1.7147196528849047
건지다 -1.6390566132458846
이상하다 -1.6377384637084662
그닥 -1.6141670820514722
많이 -1.60638668152865
비싸다 -1.6047071239650885
비추다 -1.5941805112839162
가격 -1.571042928698737
맞다 -1.545329947295077
써주다 -1.544556431322983
원래 -1.5335686328493967
보통 -1.5278537294224632
시어 -1.5185523612224514
밀가루 -1.5139030796777493
적어지다 -1.5113072221558976
아주아주 -1.4955271914380246
개인 -1.495043337639411
닥 -1.4896204317060515
비다 -1.4535437915801035
싱겁다 -1.4411578199929653
아스파탐 -1.4399599926430084
스럽다 -1.40